In [13]:
PROCESSED_DATA_FP = '../data/boston/processed/'
MAP_FP = PROCESSED_DATA_FP + 'maps/'
import sys
import json

In [14]:
import os
sys.path.append('../src')
BASE_DIR = os.path.dirname(os.getcwd())
print(BASE_DIR)

print(os.getcwd())
from data import util

/home/jenny/boston-crash-modeling
/home/jenny/boston-crash-modeling/notebooks


In [15]:
items = json.load(open(PROCESSED_DATA_FP + 'crash_joined.json'))
crash_list, crashes_by_id = util.group_json_by_location(items)
print(str(len(crash_list)) + " crashes found")


# Crashes by id is a dict where the key is the segment id.  It contains the count of crashes at that intersection
# Can also contain other information about this location, if called with arg otherfields.  In this case, we're
# Looking at time
sorted = [value['count'] for key,value in crashes_by_id.items() if key != '']
sorted.sort(reverse=True)
print(str(len(sorted)) + " unique crash locations found\n")

print("Highest number of crashes at any one location: " + str(sorted[0:10]))
worst_id = [key for key, value in crashes_by_id.items() if value['count'] == sorted[0]][0]


12967 crashes found
4903 unique crash locations found

Highest number of crashes at any one location: [55, 52, 49, 43, 42, 42, 41, 39, 38, 32]


In [16]:
def get_int_feat(featname, items, buckets=[]):
    results = {}
    for id, values in items.items():
        value = max([x[featname] for x in values])
        
        if buckets:
            found = False
            for bucket in buckets:
                if int(value) <= bucket and not found:
                    value = bucket
                    found = True
            if not found:
                value = buckets[len(buckets)-1]
        if value not in results.keys():
            results[value] = {'crash': 0, 'no_crash': 0}
        if id in crashes_by_id.keys():
            results[value]['crash'] += 1
        else:
            results[value]['no_crash'] += 1
 
    print("\n" + featname + "..............")
    for key, value in results.items():
        print(str(key) + ':' + str(float(value['crash'])/(float(value['no_crash'] + value['crash']))) + " out of " + str(value['no_crash'] + value['crash']))


In [17]:
items = json.load(open(PROCESSED_DATA_FP + '/inters_data.json'))

#feats = ['hwy_type', 'width', 'lanes', 'osm_speed', 'signal', 'SPEEDLIMIT', 'Struct_Cnd', 'Surface_Tp', 'F_F_Class']

print(len(items))
print("************************************")
get_int_feat('signal', items)
get_int_feat('lanes', items)
get_int_feat('width', items, [9, 14, 19, 29])
get_int_feat('oneway', items)

# Highway types are kind of problematic to use max for, but it looks like if they're all residential roads, that really reduces crash risk
# Also, too many highway types with combo types (e.g. motorway and motorway link...)
get_int_feat('hwy_type', items)

get_int_feat('SPEEDLIMIT', items)

get_int_feat('Struct_Cnd', items)

get_int_feat('Surface_Tp', items)

get_int_feat('F_F_Class', items)

7790
************************************

signal..............
0:0.1959924859110833 out of 4791
1:0.4851617205735245 out of 2999

lanes..............
2:0.26707083599234205 out of 6268
3:0.5409582689335394 out of 647
4:0.5988023952095808 out of 501
5:0.6666666666666666 out of 75
1:0.04938271604938271 out of 162
6:0.6666666666666666 out of 6
0:0.061068702290076333 out of 131

width..............
19:0.3655813953488372 out of 2150
29:0.5483091787439613 out of 2070
14:0.14216944801026957 out of 3116
9:0.06607929515418502 out of 454

oneway..............
0:0.22323049001814882 out of 2755
1:0.3533267130089374 out of 5035

hwy_type..............
0:0.11414326948307531 out of 3811
1:0.5288888888888889 out of 1125
2:0.6473429951690821 out of 207
3:0.5080269290523045 out of 1931
6:0.3560606060606061 out of 132
5:0.23931623931623933 out of 117
8:0.4305555555555556 out of 144
11:0.2358490566037736 out of 106
9:0.4 out of 75
12:0.39655172413793105 out of 58
4:0.2727272727272727 out of 22
7:0.3513513

In [20]:
get_int_feat('intersection_segments', items, [3, 4, 5, 6, 10, 15, 20, 40])
get_int_feat('width_per_lane', items, [5, 10, 15, 20, 25, 50])


intersection_segments..............
4:0.31384892086330934 out of 2224
10:0.5272496831432193 out of 789
6:0.40160642570281124 out of 498
15:0.5942857142857143 out of 175
5:0.3403010033444816 out of 1196
3:0.1904427266338721 out of 2846
40:0.2916666666666667 out of 24
20:0.5263157894736842 out of 38

width_per_lane..............
10:0.27605381165919285 out of 5575
15:0.4585551330798479 out of 1315
5:0.09090909090909091 out of 517
25:0.5194805194805194 out of 77
20:0.5856573705179283 out of 251
50:0.32727272727272727 out of 55
